<h2>Ping Results for cooperative</h2>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import figure
from datetime import datetime
import numpy as np
import math
import os

path = os.path.abspath("C:/Users/steve/Google Drive/Master/Masterarbet/Evaluation/graphics/disruption")

font = {'size': 13}
plt.rc('font', **font)


pingDataVM1Both = pd.read_csv('pingVM1ResultsMTDBoth.txt')
pingDataVM2Both = pd.read_csv('pingVM2ResultsMTDBoth.txt')
pingDataVM1Both = pingDataVM1Both.fillna(-1)
pingDataVM2Both = pingDataVM2Both.fillna(-1)



#print(pingDataVM1Both)
#print(pingDataVM2Both)

In [ ]:

numOfInfectionsOFVM1 = pingDataVM1Both.iloc[-1,-1]

def getSeconds(wholedf):
    pingVM1 = []
    flag = False
    timeDiffs = []
    resultVM1Infected = []
    resultSecondsVM1Infected = []

    numOfInfections = wholedf.iloc[-1,-1]
    for infection in range(1,int(numOfInfections+1)):
        pingVM1 = []
        #print(infection)
        #print(type(wholedf["bashliteRun"][40]))
        df = wholedf.loc[wholedf['bashliteRun'] == int(infection)]
        df = df.reset_index(drop=True)
        #print(df)
        #print(df)
        firstHelp = False
        secondHelp = False
        flag = False
        for index, row in df.iterrows():
            
            # get the time before the IP change starts 
            if index != df.index[-1]:
                if flag == False and df.iloc[index+1]["loss"] == -1:
                    #print("row", row)
                    pingVM1.append(datetime.strptime(df.iloc[index+1]["time"],'%H:%M:%S'))
                    #print("pingVM1 1, ", pingVM1)
                    flag = True
                    firstHelp = True

                # get the first time after the IP change
                if firstHelp == True and df.iloc[index+1]["loss"] != -1: 
                    #print("row", row)
                    #print(datetime.strptime(row["time"], '%H:%M:%S'))
                    pingVM1.append(datetime.strptime(row["time"], '%H:%M:%S'))
                    #print("pingVM1 2, ", pingVM1)
                    firstHelp = False
                    flag = False
            
                
             
        infected = [b - a for a, b in zip(pingVM1[::2], pingVM1[1::2])]
        resultSecondsVM1Infected = [x.seconds for x in infected]
        #print("resultSecondsVM1Infected", resultSecondsVM1Infected)    
        
        
        resultVM1Infected.append(sum(resultSecondsVM1Infected))
                    
        #print(pingVM1)
        #print("\n\n")
        timeStart = df.iloc[0]["time"]
        #print("timeStart", timeStart)
        

        
        timeEnd = df.iloc[-1]["time"]
        timeDiff = pd.to_datetime(timeEnd) - pd.to_datetime(timeStart)
        timeDiffs.append(timeDiff.total_seconds())
        #print("timeDiffs", timeDiffs)
    
    #print(resultVM1Infected)
        
   
    return (resultVM1Infected,timeDiffs)


def getSecondsForVM2(resultSecondsVM1,pingDataVM2Both):
    timeDiffs = []
    #print(numOfInfections)

    #print(len(pingDataVM2Both))
    numToDivide = len(pingDataVM2Both)/numOfInfectionsOFVM1
    numToDivide = math.ceil(numToDivide)

    for i in range(0,numOfInfectionsOFVM1):
        #print("new i", i)
        start = i*numToDivide 
        end = (i+1)*numToDivide
        df = pingDataVM2Both.iloc[start:end]
        timeStart = df.iloc[0]["time"]
        timeEnd = df.iloc[-1]["time"]
        timeDiff = pd.to_datetime(timeEnd) - pd.to_datetime(timeStart)
        timeDiffs.append(timeDiff.total_seconds())
        
    
    
    resultSecondsVM2Infected = [0] * numOfInfectionsOFVM1
    return (resultSecondsVM2Infected,timeDiffs) 


resultSecondsVM1 = getSeconds(pingDataVM1Both)
resultSecondsVM1 = (resultSecondsVM1[0][-27:-1],resultSecondsVM1[1][-27:-1])
resultSecondsVM2 = getSecondsForVM2(resultSecondsVM1,pingDataVM2Both)

print(resultSecondsVM1)
print(resultSecondsVM2)








In [ ]:
import matplotlib.pyplot as plt
import numpy as np


overallInfected = []
overallTime = []

for i in range(len(resultSecondsVM1[0])):
    overallInfected.append(resultSecondsVM1[0][i] + resultSecondsVM2[0][i])
    overallTime.append(resultSecondsVM1[1][i] + resultSecondsVM2[1][i])

print(overallInfected)
print(overallTime)

labelsBoth = []
for i in range(len(resultSecondsVM1[0])):
    labelsBoth.append(i)



width = 0.35       # the width of the bars: can also be len(x) sequence
fig, ax = plt.subplots(figsize=(10,6))


meanBoth = sum(overallInfected)/len(overallInfected)
    
print(f"The mean duration the : {meanBoth}")

#overallTime = [250] * 32


ax.bar(labelsBoth,overallInfected, width,label="Duration of sendable outgoing Packets")
ax.bar(labelsBoth,overallTime, width, bottom=overallInfected,
       label='Duration of outgoing Packet Loss')

ax.axhline(meanBoth, color='r', linestyle='--', label='Mean Duration of outgoing Packet Loss')
plt.ylim(0, 300)
plt.xticks(range(0, 26, 2))
plt.title('Outgoing Packet Loss on both VMs with cooperative and reactive MTD')
plt.xlabel('Current Epoch')
plt.ylabel('Seconds')
ax.legend()

plt.show()

In [ ]:
labelsBoth = "Outgoing Packet Loss", "Outgoing Packet Success"
sumInfectedBoth = sum(overallInfected)
sumTimeBoth = sum(overallTime)

shareBoth = (sumInfectedBoth/sumTimeBoth)*100

sizes = [shareBoth,100-shareBoth]
colors = ["#d62728", "#1f77b4"]
width = 0.35
fig1, ax1 = plt.subplots(figsize=(10,6))
fig1.patch.set_facecolor('white')
ax1.pie(sizes, labels=labelsBoth, autopct='%1.1f%%', startangle=177,colors = colors, textprops=dict(color="black"),labeldistance=None)
ax1.axis('equal')

ax1.legend(labelsBoth,title='')
ax1.set_title("Average Share of Packet Loss on both VMs with cooperative and reactive MTD")


filename = "packetLossCoopAndReactive.png"
plt.savefig(os.path.join(path,filename),dpi= 800)

plt.show()

<h2>Ping Results for not cooperative</h2>

In [ ]:
pingDataVM1OnlyIP = pd.read_csv('pingVM1ResultsMTDOnlyIP.txt')
pingDataVM2OnlyIP = pd.read_csv('pingVM2ResultsMTDOnlyIP.txt')
pingDataVM1OnlyIP = pingDataVM1OnlyIP.fillna(-1)
pingDataVM2OnlyIP = pingDataVM2OnlyIP.fillna(-1)

#print(pingDataVM1OnlyIP)
#print(pingDataVM2OnlyIP)

In [ ]:
resultSecondsVM1 = getSeconds(pingDataVM1OnlyIP)
resultSecondsVM2 = getSeconds(pingDataVM2OnlyIP)

print(resultSecondsVM1)
print(resultSecondsVM2)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


overallInfected = []
overallTime = []

print(len(resultSecondsVM1[0]))
print(len(resultSecondsVM1[1]))
print(len(resultSecondsVM2[0]))
print(len(resultSecondsVM2[1]))

resultSecondsVM1 = (resultSecondsVM1[0][-27:-1],resultSecondsVM1[1][-27:-1])
resultSecondsVM2 = (resultSecondsVM2[0][-27:-1],resultSecondsVM2[1][-27:-1])


#bashlite does sometimes fail to infect, take care of that here
for i in range(len(resultSecondsVM2[0])):
    if resultSecondsVM2[1][i] > 140:
        if not resultSecondsVM2[0][i] > 8:
            resultSecondsVM2[1][i]/=2
            
for i in range(len(resultSecondsVM1[0])):
    if resultSecondsVM1[1][i] > 140:
        if resultSecondsVM1[0][i] > 7:
            resultSecondsVM1[1][i]/=2
            resultSecondsVM1[0][i]/=2

print("resultSecondsVM2", resultSecondsVM2)
        
    
for i in range(len(resultSecondsVM2[0])):
    overallInfected.append(resultSecondsVM1[0][i] + resultSecondsVM2[0][i])
    overallTime.append(resultSecondsVM1[1][i] + resultSecondsVM2[1][i])

print(overallInfected)
print(overallTime)

labels = []
for i in range(len(resultSecondsVM2[0])):
    labels.append(i)



width = 0.35       # the width of the bars: can also be len(x) sequence
fig, ax = plt.subplots(figsize=(10,6))


meanOnlyIP = sum(overallInfected)/len(overallInfected)
    
print(f"The mean duration the : {meanOnlyIP}")

ax.bar(labels,overallInfected, width,label="Duration of sendable outgoing Packets")
ax.bar(labels,overallTime, width, bottom=overallInfected,
       label='Duration of outgoing Packet Loss')


ax.axhline(meanOnlyIP, color='r', linestyle='--', label='Mean Duration of outgoing Packet Loss')
plt.ylim(0, 300)
plt.xticks(range(0, 26, 2))
plt.title('Outgoing Packet Loss on both VMs with non-cooperative and reactive MTD')
plt.xlabel('Current Epoch')
plt.ylabel('Seconds')
ax.legend(loc=1)

plt.show()

In [ ]:
labelsBoth = "Outgoing Packet Loss", "Outgoing Packet Success"
sumInfectedOnlyIP = sum(overallInfected)
sumTimeOnlyIP = sum(overallTime)

shareOnlyIP = (sumInfectedOnlyIP/sumTimeOnlyIP)*100

sizes = [shareOnlyIP,100-shareOnlyIP]
colors = ["#d62728", "#1f77b4"]
width = 0.35       # the width of the bars: can also be len(x) sequence
fig1, ax1 = plt.subplots(figsize=(10,6))
fig1.patch.set_facecolor('white')
ax1.pie(sizes, labels=labelsBoth, autopct='%1.1f%%', startangle=175,colors = colors, textprops=dict(color="black"),labeldistance=None)
ax1.axis('equal')
ax1.legend(labelsBoth,title='')
ax1.set_title("Average Share Packet Loss on both VMs with non-cooperative and reactive MTD")

filename = "packetLossNoncoopAndReactive.png"
plt.savefig(os.path.join(path,filename),dpi= 800)

plt.show()




<h2>Code for Cooperative but Active</h2>

In [ ]:
pingDataVM1BothActive = pd.read_csv('pingVM1ResultsMTDBothActive.txt')
pingDataVM2BothActive = pd.read_csv('pingVM2ResultsMTDBothActive.txt')
pingDataVM1BothActive = pingDataVM1BothActive.fillna(-1)
pingDataVM2BothActive = pingDataVM2BothActive.fillna(-1)

print(pingDataVM1BothActive)
print(pingDataVM2BothActive)

In [ ]:
import math


def getSecondsActive(wholedf):
    pingVM1 = []
    flag = False
    timeDiffs = []
    resultVM1Infected = []
    resultSecondsVM1Infected = []

    numOfInfections = wholedf.iloc[-1,-1]
    for infection in range(1,int(numOfInfections+1)):
        pingVM1 = []
        df = wholedf.loc[wholedf['bashliteRun'] == int(infection)]
        df = df.reset_index(drop=True)
        #print(df)
        firstHelp = False
        secondHelp = False
        flag = False
        for index, row in df.iterrows():
            
            # get the time before the IP change starts 
            if index != df.index[-1]:
                if flag == False and df.iloc[index+1]["loss"] == -1:        
                    #print("row first", row)
                    pingVM1.append(datetime.strptime(df.iloc[index+1]["time"],'%H:%M:%S'))
                    #print("pingVM1 1, ", pingVM1)
                    flag = True
                    firstHelp = True

                # get the first time after the IP change
                if firstHelp == True and (df.iloc[index+1]["loss"] != -1): 
                    #print("row last", row)
                    #print("row", row)
                    #print(datetime.strptime(row["time"], '%H:%M:%S'))
                    pingVM1.append(datetime.strptime(row["time"], '%H:%M:%S'))
                    #print("pingVM1 2, ", pingVM1)
                    firstHelp = False
                    flag = False
            
            elif index == df.index[-1] and row["loss"] == -1:
                helperdf = wholedf.loc[wholedf['bashliteRun'] == int(infection)+1]
                if helperdf.iloc[0]["bashliteRun"] == row["bashliteRun"] + 1:
                    pingVM1.append(datetime.strptime(helperdf.iloc[0]["time"],'%H:%M:%S'))

                    #print("row last", row)
                    firstHelp = False
                    flag = False
                
                
            
                
             
        infected = [b - a for a, b in zip(pingVM1[::2], pingVM1[1::2])]
        resultSecondsVM1Infected = [x.seconds for x in infected]
        #print("resultSecondsVM1Infected", resultSecondsVM1Infected)    
        
        
        resultVM1Infected.append(sum(resultSecondsVM1Infected))
                    
        #print(pingVM1)
        #print("\n\n")

        timeStart = df.iloc[0]["time"]
        #print("timeStart", timeStart)
        

        
        timeEnd = df.iloc[-1]["time"]
        timeDiff = pd.to_datetime(timeEnd) - pd.to_datetime(timeStart)
        timeDiffs.append(timeDiff.total_seconds())
        #print("timeDiffs", timeDiffs)
        #print(df)
    
    #print(resultVM1Infected)
    #print("timeDiffs", timeDiffs )
        
   
    return (resultVM1Infected,timeDiffs)


def getSecondsForVM2Active(resultSecondsVM1,pingDataVM2Both):
    timeDiffs = []
    numOfInfections = len(resultSecondsVM1[0])
    #print(numOfInfections)

    #print(len(pingDataVM2Both))
    numToDivide = len(pingDataVM2Both)/numOfInfections
    numToDivide = math.ceil(numToDivide)

    for i in range(0,numOfInfections):
        #print("new i", i)
        start = i*numToDivide 
        end = (i+1)*numToDivide
        df = pingDataVM2Both.iloc[start:end]
        timeStart = df.iloc[0]["time"]
        timeEnd = df.iloc[-1]["time"]
        timeDiff = pd.to_datetime(timeEnd) - pd.to_datetime(timeStart)
        timeDiffs.append(timeDiff.total_seconds())
        
    

    resultSecondsVM2Infected = [0] * numOfInfections
    return (resultSecondsVM2Infected,timeDiffs) 


resultSecondsVM1Active = getSecondsActive(pingDataVM1BothActive)
resultSecondsVM2Active = getSecondsForVM2Active(resultSecondsVM1Active,pingDataVM2BothActive)

resultSecondsVM1Active = (resultSecondsVM1Active[0][-27:-1],resultSecondsVM1Active[1][-27:-1])
resultSecondsVM2Active = (resultSecondsVM2Active[0][-27:-1],resultSecondsVM2Active[1][-27:-1])

#print(resultSecondsVM1Active)
#print(resultSecondsVM2Active)








In [ ]:

import matplotlib.pyplot as plt
import numpy as np


overallInfectedActive = []
overallTimeActive = []

for i in range(len(resultSecondsVM1Active[0])):
    overallInfectedActive.append(resultSecondsVM1Active[0][i] + resultSecondsVM2Active[0][i])
    overallTimeActive.append(resultSecondsVM1Active[1][i] + resultSecondsVM2Active[1][i])

print(overallInfectedActive)
print(overallTimeActive)

labelsBoth = []
for i in range(len(resultSecondsVM1Active[0])):
    labelsBoth.append(i)



width = 0.35       # the width of the bars: can also be len(x) sequence
fig, ax = plt.subplots(figsize=(10,6))


meanBoth = sum(overallInfectedActive)/len(overallInfectedActive)
    
print(f"The mean duration the : {meanBoth}")



ax.bar(labelsBoth,overallInfectedActive, width,label="Duration of sendable outgoing Packets")
ax.bar(labelsBoth,overallTimeActive, width, bottom=overallInfectedActive,
       label='Duration of outgoing Packet Loss')

ax.axhline(meanBoth, color='r', linestyle='--', label='Mean Duration of outgoing Packet Loss')
plt.ylim(0, 400)
plt.xticks(range(0, 20, 2))
plt.title('Outgoing Packet Loss on both VMs with cooperative and proactive MTD')
plt.xlabel('Current Epoch')
plt.ylabel('Seconds')
ax.legend()

plt.show()

In [ ]:
labelsBoth = "Outgoing Packet Loss", "Outgoing Packet Success"
sumInfectedBothActive = sum(overallInfectedActive)
sumTimeBothActive = sum(overallTimeActive)

shareBothActive = (sumInfectedBothActive/sumTimeBothActive)*100

sizes = [shareBothActive,100-shareBothActive]
colors = ["#d62728", "#1f77b4"]
width = 0.35
fig1, ax1 = plt.subplots(figsize=(10,6))
fig1.patch.set_facecolor('white')
ax1.pie(sizes, labels=labelsBoth, autopct='%1.1f%%', startangle=175,colors = colors,textprops=dict(color="black"),labeldistance=None)
ax1.axis('equal')
ax1.legend(labelsBoth,title='')
ax1.set_title("Outgoing Packet Loss on both VMs with cooperative and proactive MTD")

filename = "packetLossCoopAndProactive.png"
plt.savefig(os.path.join(path,filename),dpi= 800)

plt.show()




In [ ]:
print(shareOnlyIP)
print(shareBoth)
print(shareBothActive)


font = {'size': 18}
plt.rc('font', **font)



width = 0.35   

fig, ax = plt.subplots(figsize=(8,10))

ax.bar(['Non-Cooperative \n (reactive)', 'Cooperative \n (reactive)','Cooperative \n (proactive)'], [shareOnlyIP, shareBoth,shareBothActive])
plt.title('Average Share of Outgoing Packet Loss')
plt.xlabel('MTD Type')
plt.ylabel('Share of Packet Loss in %')


plt.tight_layout()
filename = "packetLossSummary.png"
plt.savefig(os.path.join(path,filename),dpi= 800)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


width = 0.35       # the width of the bars: can also be len(x) sequence
fig, ax = plt.subplots(figsize=(10,6))



linfections = [i for i in range(0, len(resultSecondsVM1))]
#print(labels)

mean = sum(data)/len(data)
    
    
mean = np.mean(summedUP)
print(f"The mean duration of both Infected is: {mean}")

ax.bar(labels,resultSecondsVM1, width,label="VM1")
ax.bar(labels,resultSecondsVM2, width, bottom=resultSecondsVM1,
       label='VM2')

ax.axhline(mean, color='r', linestyle='--', label='mean')
plt.ylim(0, 25)
plt.xticks(range(0, 20, 2))
plt.title('Duration of Bashlite on both VMs only with IP Change')
plt.xlabel('Bashlite Runs')
plt.ylabel('Duration of Bashlite in Seconds')
ax.legend()

plt.show()

In [ ]:

def getSeconds(df):
    pingVM1 = []
    flag = False
    for index, row in df.iterrows():
        #print(row["time"])
        #print(pingDataVM1Both.iloc[index+1]["loss"])
        try:
            if flag == False and df.iloc[index+1]["loss"] == -1:
                #print(flag)
                #print(row)
                pingVM1.append(row["time"])
                flag = True
            if flag == True and df.iloc[index+1]["loss"] == "0%":
                pingVM1.append(df.iloc[index]["time"])
                flag = False                      
        except IndexError:
            print("finito")
            
    print(pingVM1)

    lastTimeOfBashRun = pingVM1[1::2]
    print(lastTimeOfBashRun)
    print(len(lastTimeOfBashRun))
    
    df["BashliteRun"] = None
    
    count = 0
    add = True
    for index, row in df.iterrows():
        if count < len(lastTimeOfBashRun):
            print(count)
            print(lastTimeOfBashRun[count])
            if row["time"] != lastTimeOfBashRun[count]:
                df["BashliteRun"] = count
            else:
                count += 1

    
    #resultVM1 = [b - a for a, b in zip(pingVM1[::2], pingVM1[1::2])]
    #resultSecondsVM1 = [x.seconds for x in resultVM1]
    #return resultSecondsVM1
    
resultSecondsVM1 = getSeconds(pingDataVM1Both)

print(resultSecondsVM1)






